# Search with base models

## Goal

Can we solve ARC tasks using base models with access to a DSL?

## Imports

In [ ]:
import os
import logging
from arc25.utils import get_least_used_gpu_index
from arc25.logging import configure_logging, log_execution_time

configure_logging()
os.environ['CUDA_VISIBLE_DEVICES'] = str(get_least_used_gpu_index())

# Add VLLM specific environment variables to avoid common issues
os.environ['VLLM_USE_MODELSCOPE'] = 'False'
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'

In [ ]:
import time
import importlib
import inspect
import json
import gc

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from vllm import LLM, SamplingParams

from arc25.training_tasks import *
from arc25.encoders import create_grid_encoder
from arc25.prompting import pretty_print_prompt, Template

## Code

### Prompt

https://github.com/flowersteam/SOAR/blob/main/soar/prompt.py

In [ ]:
def extract_footprint(module_name: str, show_types: bool = False) -> str:
    """
    Load a module by name, then return a newline-separated list of all
    top-level functions in it, in the form:

      def func_name(arg1, arg2) -> return

    If show_types=True, annotations are included; otherwise only names.
    """
    mod = importlib.import_module(module_name)
    footprints = []

    for name, fn in inspect.getmembers(mod, inspect.isfunction):
        # skip imports from elsewhere
        if fn.__module__ != module_name or name.startswith("_"):
            continue

        sig = inspect.signature(fn)
        if not show_types:
            # strip type info
            params = [p.name for p in sig.parameters.values()]
            sig_text = f"({', '.join(params)})"
        else:
            sig_text = str(sig)

        footprints.append(f"- {name}{sig_text}")

    return "\n".join(footprints)

print(extract_footprint('arc25.BARC_dsl', show_types=True))

In [ ]:
with open('/mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json', 'r') as f:
    training_challenges = json.load(f)

def get_task(task_name):
    if task_name in training_challenges:
        task_data = training_challenges[task_name]
        inputs = [Img(sample['input']) for sample in task_data['train']]
        outputs = [Img(sample['output']) for sample in task_data['train']]
        return Task(inputs=inputs, outputs=outputs, code='', name=task_name)
    raise ValueError(f"Task {task_name} not found in training challenges.")

In [ ]:
system_prompt = """You are an advanced AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks."""


prompt_template = Template("""You are tasked with solving a transformation problem from the Abstraction and Reasoning Challenge (ARC).
Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ```). You must write a function called `transform` which takes a single argument, the input grid as `list[list[int]]`, and returns the transformed grid (also as `list[list[int]]`).

## Key Priors:

- **Objectness**: Consider the grid as containing objects (groups of connected cells) rather than just individual pixels.
- **Goal-Directed**: The transformation should achieve a specific goal, such as creating symmetry or changing the color of specific objects.
- **Numbers & Counting**: Keep track of the number of objects, sizes, and their relative positions.
- **Geometry & Topology**: Use spatial relationships such as adjacency, enclosure, or symmetry.

Carefully analyze the examples and find the underlying transformation logic.

## Domain Specific Primitive Functions

You can use the already implemented following functions to manipulate the grid:

{{ dsl }}

## Examples

Below are several input-output examples that illustrate the transformation.
Your function should generalize the pattern from these examples to solve any input following the same logic.

{% for sample in train_samples %}
### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}
{% endfor %}
""")


def create_prompt_from_task(task, grid_encoder, tokenizer):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True)
    return prompt

### Model

In [ ]:
@log_execution_time
def load_model(base_model_path, lora_path=None, quantize_4bit=True):
    logging.info(f"Loading model from {base_model_path} and LoRA from {lora_path}")
    torch.cuda.empty_cache()

    if quantize_4bit:
        bnb_config = BitsAndBytesConfig(
                load_in_4bit= True,
                bnb_4bit_quant_type= "nf4",
                bnb_4bit_compute_dtype= torch.float16,
                bnb_4bit_use_double_quant= True,
                llm_int8_enable_fp32_cpu_offload= True,
                llm_int8_skip_modules=['gate', 'lm_head'],
        )
    else:
        bnb_config = None


    model = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype="auto", device_map="auto", quantization_config=bnb_config)
    if lora_path is None:
        tokenizer = AutoTokenizer.from_pretrained(base_model_path)
        return model, tokenizer

    tokenizer = AutoTokenizer.from_pretrained(lora_path)
    model = PeftModel.from_pretrained(model, lora_path, is_trainable=True)
    return model, tokenizer


def cleanup_model(model=None):
    """Clean up GPU memory before loading VLLM"""
    if model is not None:
        del model
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

In [ ]:
raise

## First steps

In [ ]:
model, tokenizer = load_model("/home/gbarbadillo/models/Qwen2.5-Coder-7B-Instruct")

In [ ]:
print(len(tokenizer.tokenize(extract_footprint('arc25.BARC_dsl', show_types=True))))
prompt = create_prompt_from_task(get_task('007bbfb7'), grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))'), tokenizer=tokenizer)
pretty_print_prompt(prompt, default_color='white')

In [ ]:
model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)
t0 = time.time()
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    do_sample=True,
    temperature=1.0,
    top_p=0.90,
    num_return_sequences=1,
)
t1 = time.time()
generated_ids = generated_ids[:, len(model_inputs.input_ids[0]):]
output_tokens = generated_ids.shape[1]
print(f"Generated {output_tokens} tokens in {t1 - t0:.2f} seconds")
print(f"Average tokens per second: {output_tokens / (t1 - t0):.2f} tokens/second")
predictions = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(predictions[0])

## VLLM

In [ ]:
model_path = "/home/gbarbadillo/models/Qwen2.5-Coder-3B-Instruct"

llm = LLM(
    model=model_path,
    gpu_memory_utilization=0.8,  # Use less GPU memory
    # max_model_len=4096,  # Limit context length
    trust_remote_code=True,
    dtype="bfloat16",  # Use float16 to save memory
    tensor_parallel_size=1,  # Single GPU
    # quantization="bitsandbytes",  # Use NF4 quantization
)

In [ ]:
task_ids = list(training_challenges.keys())[:10]
vllm_tokenizer = AutoTokenizer.from_pretrained(model_path)
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')
prompts = [create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder, tokenizer=vllm_tokenizer) for task_id in task_ids]

In [ ]:
sampling_params = SamplingParams(temperature=1.0, top_p=0.95, max_tokens=1024)
t0 = time.time()
outputs = llm.generate(prompts, sampling_params)
total_tokens = sum(len(output.outputs[0].token_ids) for output in outputs)
t1 = time.time()
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {t1 - t0:.2f} seconds")
print(f"Average time per task: {(t1 - t0) / len(outputs):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(outputs):.2f} tokens")
print(f"Average tokens per second: {total_tokens / (t1 - t0):.2f} tokens/second")

In [ ]:
print("Generated output:")
for task_id, output in zip(task_ids, outputs):
    print('*'*80, f"Task ID: {task_id}", '*'*80, '', sep='\n')
    print(output.outputs[0].text)

- 3B quantized model: 
- 7B quantized model: 2.5s per task

```
0.5 unquantized
Total tokens generated: 6672
Time taken: 6.02 seconds
Average time per task: 0.60 seconds
Average tokens per task: 667.20 tokens
Average tokens per second: 1107.48 tokens/second

0.5B
Total tokens generated: 4493
Time taken: 7.97 seconds
Average time per task: 0.80 seconds
Average tokens per task: 449.30 tokens
Average tokens per second: 563.84 tokens/second

1.5B unquantized
Total tokens generated: 1956
Time taken: 4.59 seconds
Average time per task: 0.46 seconds
Average tokens per task: 195.60 tokens
Average tokens per second: 426.54 tokens/second

1.5B
Total tokens generated: 2805
Time taken: 9.93 seconds
Average time per task: 0.99 seconds
Average tokens per task: 280.50 tokens
Average tokens per second: 282.56 tokens/second

3B unquantized
Total tokens generated: 5465
Time taken: 12.09 seconds
Average time per task: 1.21 seconds
Average tokens per task: 546.50 tokens
Average tokens per second: 452.00 tokens/second

3B
Total tokens generated: 5689
Time taken: 31.51 seconds
Average time per task: 3.15 seconds
Average tokens per task: 568.90 tokens
Average tokens per second: 180.54 tokens/second

7B
Total tokens generated: 2785
Time taken: 22.92 seconds
Average time per task: 2.29 seconds
Average tokens per task: 278.50 tokens
Average tokens per second: 121.53 tokens/second

Total tokens generated: 4053
Time taken: 56.17 seconds
Average time per task: 5.62 seconds
Average tokens per task: 405.30 tokens
Average tokens per second: 72.15 tokens/second
```

## TODO

- [x] Create a prompt with the available DSL functions and the training ARC task
- [x] Fix VLLM initialization issues with proper memory management
- [ ] Verify the effect of caching
- [ ] Update the library to be able to select which DSL to use when executing code
- [ ] Try to solve some easy task
- [ ] Create a refine prompt
- [ ] Make a more complex tree search